In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('matrice_votes_deputes.csv')

In [3]:
df

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10
0,thibault-bazin,0,1,-1,-1,-1,-1,-1,-1,-1,-1
1,marc-fesneau,1,2,2,2,2,2,2,2,2,-1
2,laurence-maillart-mehaignerie,1,2,2,2,2,2,2,2,2,2
3,olivier-veran,1,2,2,-1,-1,-1,-1,-1,-1,-1
4,daniele-obono,-1,-1,1,1,1,1,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...
264,pierre-cordier,0,2,2,2,2,2,2,2,-1,2
265,nathalie-bassire,0,2,2,2,2,2,2,2,2,2
266,jean-luc-warsmann,1,2,2,2,2,2,2,2,2,2
267,david-habib,1,2,2,2,1,1,1,1,2,2
